In [3]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
from library.correlationAndTtestLib import *

In [4]:
def time_zone_cal(s):
    '''
    @param s: a string of format "hh:mm:ss"
    
    @return: a string denoting the timezone
    '''
    hour=int(s.split(':')[0])

    if 6<hour<=9:
        time_zone='Early_Morning'
    elif 9<hour<=12:
        time_zone='Morning'
    elif 12<hour<=17:
        time_zone='Afternoon'
    elif 17<hour<=23:
        time_zone='Evening'
    return time_zone

# converts hour:minute:seconds to seconds
def time_as_int(timestr):
    '''
    @param timestr: a string of format "hh:mm:ss"
    
    @return: an integer denoting the time in seconds
    '''
    hh, mm, ss = timestr.split(":")
    time_int = (int(hh)*3600)+(int(mm)*60)+int(ss)
    return time_int

def get_day_of_week(date_string, seperator="/", date_format="mm/dd/yyyy"):
    '''
    @param date_string: a string denoting a date
    @param seperator (optional): a character which acts as seperator in the date_string. Default = "/"
    @param date_format (optional): a string which denotes the format the date string is. Default = "mm/dd/yyyy"
                                Supported values are : {"dd/mm/yyyy", "mm/dd/yyyy"}
    
    @return: a string denoting the day of week corressponding to the date denoted by date_string
    '''
    date_format_idxs = {'mm/dd/yyyy':{'date_idx': 1, 'month_idx': 0, 'year_idx': 2},
                        'dd/mm/yyyy':{'date_idx': 0, 'month_idx': 1, 'year_idx': 2}}
    try:
        idx_dict = date_format_idxs[date_format]
        date_arr = [int(val) for val in date_string.split(seperator)]
        weekdays = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
        date = datetime.datetime(date_arr[idx_dict['year_idx']],\
                                 date_arr[idx_dict['month_idx']],\
                                 date_arr[idx_dict['date_idx']])
        return weekdays[date.weekday()]
    except KeyError:
        raise ValueError(f"{date_format} is not supported. Supported values are: {list(date_format_idxs.keys())}.")

In [5]:
zones = ["junct_mall", "prantika_bus_stand"]

In [6]:
log_dir = r"../reports/54ft/"
instance_wise_scores_dir = "instance_wise_scores"

dates = ["14_09_2019", "15_09_2019", "16_09_2019", "17_09_2019", "20_09_2019", "21_09_2019", "23_09_2019"]

## 1

In [7]:
zone_timezone_analysis_dfs = []
for zone in zones:
    path = os.path.join(log_dir, zone, f"time_zone_analysis/score_timezone_analysis_{zone}_zone.csv")
    zone_timezone_analysis_dfs.append(pd.read_csv(path))

In [8]:
array_1, array_2 = zone_timezone_analysis_dfs[0]['Whole_Day'], zone_timezone_analysis_dfs[1]['Whole_Day']
zonal_welch_t_result = welch_t_test(array_1, array_2)
print(f"Welch's (t_test_stat, p_value) for zones {zones}: {zonal_welch_t_result}")

Welch's (t_test_stat, p_value) for zones ['junct_mall', 'prantika_bus_stand']: (6.9628689398734664, 2.0936879865449384e-05)


## 2.A

In [9]:
score_dfs = {}

for zone in zones:
    for date in dates:
        score_dfs[zone] = score_dfs.get(zone, {})
        score_dfs[zone][date] = pd.read_csv(os.path.join(log_dir, zone, 
                                                      instance_wise_scores_dir, 
                                                      f"{date}.csv"))

In [10]:
# test
for zone in score_dfs.keys():
    for date in dates:
        temp_df = score_dfs.get(zone).get(date)
        temp_df['Date'] = [date for i in range(len(temp_df))]

In [11]:
joined_dfs = {}
for zone in score_dfs.keys():
    temp_df = pd.concat(score_dfs[zone].values(), ignore_index=True)
    joined_dfs[zone] = temp_df
    print(len(temp_df))

53
54


In [12]:
for temp_df in joined_dfs.values():
    temp_df['Timezone'] = temp_df['start_time'].apply(lambda x : time_zone_cal(x))
    temp_df['Day'] = temp_df['Date'].apply(lambda x : get_day_of_week(x, seperator="_", date_format="dd/mm/yyyy"))
#     print(temp_df)

In [13]:
for zone in joined_dfs.keys():
    print(f"For {zone}: ")
    grouped = joined_dfs[zone].groupby(["Date", "Timezone"])
    for date in dates:
        print(f"\tFor {date}:")
        welch_test_pair = []
        for pair in grouped.groups.keys():
            if date in pair:
                welch_test_pair.append(pair)
                if len(welch_test_pair) == 2:
                    break
        if not len(welch_test_pair) < 2:
            pair_1, pair_2 = welch_test_pair
            array_1 = list(grouped.get_group(pair_1)['score'])
            array_2 = list(grouped.get_group(pair_2)['score'])
            
            print(f"\t\t{pair_1[1]}, {pair_2[1]}: {len(array_1)},{len(array_2)}")
            print(f"\t\t{pair_1[1]}, {pair_2[1]}:"\
                +f"{welch_t_test(array_1, array_2)}")
        else:
            print(f"\t\tOnly {len(welch_test_pair)} timezone available:\t{welch_test_pair}.")
        print()

For junct_mall: 
	For 14_09_2019:
		Afternoon, Morning: 2,3
		Afternoon, Morning:(1.0, 0.42264973081037427)

	For 15_09_2019:
		Only 1 timezone available:	[('15_09_2019', 'Afternoon')].

	For 16_09_2019:
		Afternoon, Evening: 7,2
		Afternoon, Evening:(0.6868028197434451, 0.6029039678713222)

	For 17_09_2019:
		Afternoon, Evening: 3,2
		Afternoon, Evening:(-1.8410785575880018, 0.1660703195925413)

	For 20_09_2019:
		Afternoon, Evening: 6,1
		Afternoon, Evening:(nan, nan)

	For 21_09_2019:
		Afternoon, Evening: 4,1
		Afternoon, Evening:(nan, nan)

	For 23_09_2019:
		Afternoon, Evening: 4,1
		Afternoon, Evening:(nan, nan)

For prantika_bus_stand: 
	For 14_09_2019:
		Only 1 timezone available:	[('14_09_2019', 'Morning')].

	For 15_09_2019:
		Only 1 timezone available:	[('15_09_2019', 'Afternoon')].

	For 16_09_2019:
		Afternoon, Evening: 8,2
		Afternoon, Evening:(1.7621447994377786, 0.12142146404520328)

	For 17_09_2019:
		Afternoon, Evening: 7,3
		Afternoon, Evening:(-0.45253200703182367,

## 2.B

In [14]:
a, b = joined_dfs.keys()
a_group = joined_dfs[a].groupby(["Date", "Timezone"])
b_group = joined_dfs[b].groupby(["Date", "Timezone"])
for timezone in set(joined_dfs[zone]['Timezone']):
    print(f"For {timezone}:\n")
    for date in dates:
        print(f"\tFor {date}:")
        pair = (date, timezone)
        a_exists = pair in a_group.groups.keys()
        b_exists = pair in b_group.groups.keys()
        if(a_exists or b_exists):
            if(a_exists and b_exists):
                a_array = list(a_group.get_group(pair)['score'])
                b_array = list(b_group.get_group(pair)['score'])
                print(f"\t\t{a}, {b}: {len(a_array)},{len(b_array)}")
                print(f"\t\t\t{welch_t_test(a_array, b_array)}")
            else:
                print(f"\t\tFor pair: {pair}, only available: " + a if a_exists else b)
        else:
            print("\t\tNothing found!")
    print()

For Evening:

	For 14_09_2019:
		Nothing found!
	For 15_09_2019:
		Nothing found!
	For 16_09_2019:
		junct_mall, prantika_bus_stand: 2,2
			(1.0, 0.49999999999999956)
	For 17_09_2019:
		junct_mall, prantika_bus_stand: 2,3
			(1.5215349135496974, 0.24270395613536028)
	For 20_09_2019:
		junct_mall, prantika_bus_stand: 1,2
			(nan, nan)
	For 21_09_2019:
		For pair: ('21_09_2019', 'Evening'), only available: junct_mall
	For 23_09_2019:
		For pair: ('23_09_2019', 'Evening'), only available: junct_mall

For Afternoon:

	For 14_09_2019:
		For pair: ('14_09_2019', 'Afternoon'), only available: junct_mall
	For 15_09_2019:
		junct_mall, prantika_bus_stand: 5,4
			(1.7179113807746669, 0.13523205557263326)
	For 16_09_2019:
		junct_mall, prantika_bus_stand: 7,8
			(2.6856187287007396, 0.01871571676774183)
	For 17_09_2019:
		junct_mall, prantika_bus_stand: 3,7
			(0.981421517218087, 0.4050183387297711)
	For 20_09_2019:
		junct_mall, prantika_bus_stand: 6,5
			(0.31311869559901456, 0.7615503986350938